# Fine Tuning Third Iteration


In [1]:
!pip install -U transformers

In [2]:
!pip install datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

In [3]:
# importing libraries
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

In [4]:
# loading matchup file
file_path = '/content/final_matchups.csv'
df_matchups = pd.read_csv(file_path)

df_matchups.head(5)

,fighter_a,fighter_b,input_text,label
0,Hamdy Abdelwahab,Ildemar Alcantara,Fighter A: Hamdy Abdelwahab | Height: 74.0 in ...,0
1,Daichi Abe,Rostem Akman,Fighter A: Daichi Abe | Height: 71.0 in | Reac...,1
2,Leon Aliu,Jailton Almeida,Fighter A: Leon Aliu | Height: 72.0 in | Reach...,1
3,John Adajar,Arnold Allen,Fighter A: John Adajar | Height: 69.0 in | Rea...,1
4,Jose Alday,Jailton Almeida,Fighter A: Jose Alday | Height: 67.0 in | Reac...,1


In [5]:
# final preparation
matchups = df_matchups[['input_text', 'label']]

train_texts, val_texts, train_labels, val_labels = train_test_split(
    matchups['input_text'].tolist(),
    matchups['label'].tolist(),
    test_size = 0.1,
    random_state = 42
)

In [6]:
# load Tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# tokenize datasets
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [7]:
# convert to HuggingFace dataset
class MatchupDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = MatchupDataset(train_encodings, train_labels)
val_dataset = MatchupDataset(val_encodings, val_labels)

In [8]:
# load model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
# define training arguments

# training_args = TrainingArguments(
#     output_dir="./results",
#     num_train_epochs=4,
#     per_device_train_batch_size=8,
#     per_device_eval_batch_size=8,
#     warmup_steps=50,
#     evaluation_strategy="epoch",
#     logging_dir="./logs",
#     logging_steps=10,
#     save_strategy="epoch",
# )

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=50,
    logging_dir="./logs",
    logging_steps=10,
)

# define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# training the model
trainer.train()

# save the model
model.save_pretrained("/content/fine_tuned_ufc_model")
tokenizer.save_pretrained("/content/fine_tuned_ufc_model")


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kinicooo1 (kinic) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.680900
20,0.686700
30,0.694800
40,0.726000
50,0.640900
60,0.560700
70,0.649300
80,0.611100
90,0.604900
100,0.689700


('/content/fine_tuned_ufc_model/tokenizer_config.json',
 '/content/fine_tuned_ufc_model/special_tokens_map.json',
 '/content/fine_tuned_ufc_model/vocab.txt',
 '/content/fine_tuned_ufc_model/added_tokens.json')

In [12]:
from sklearn.metrics import accuracy_score, classification_report

In [13]:
# Evaluation: Accuracy and Classification Report

# Get predictions on validation set
predictions = trainer.predict(val_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1)
labels = torch.tensor(predictions.label_ids)

# Calculate accuracy
accuracy = accuracy_score(labels, preds)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.7300


In [14]:
# Detailed classification report
print(classification_report(labels, preds, target_names=["Fighter A wins", "Fighter B wins"]))

                precision    recall  f1-score   support

Fighter A wins       0.73      0.57      0.64        42
Fighter B wins       0.73      0.84      0.78        58

      accuracy                           0.73       100
     macro avg       0.73      0.71      0.71       100
  weighted avg       0.73      0.73      0.72       100



In [15]:
# Inference Script: Predict Winner

def predict_winner(fighter_a_stats, fighter_b_stats, model, tokenizer):
    input_text = (
        f"Fighter A: {fighter_a_stats} || Fighter B: {fighter_b_stats}"
    )
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    pred = torch.argmax(probs, dim=1).item()
    return {"Fighter A wins": float(probs[0][0]), "Fighter B wins": float(probs[0][1])}, pred

In [16]:
fighter_a = "Height: 75 in | Reach: 84 in | Str. Acc: 0.57 | Str. Def: 0.58 | SLpM: 4.25 | SApM: 2.12"
fighter_b = "Height: 70 in | Reach: 71 in | Str. Acc: 0.49 | Str. Def: 0.55 | SLpM: 4.00 | SApM: 3.00"

probs, winner = predict_winner(fighter_a, fighter_b, model, tokenizer)
print(probs, "Winner Label (0=A, 1=B):", winner)

{'Fighter A wins': 0.25795623660087585, 'Fighter B wins': 0.7420437335968018} Winner Label (0=A, 1=B): 1


In [17]:
# Zip and Download
!zip -r /content/fine_tuned_ufc_model.zip /content/fine_tuned_ufc_model
from google.colab import files
files.download('/content/fine_tuned_ufc_model.zip')

  adding: content/fine_tuned_ufc_model/ (stored 0%)
  adding: content/fine_tuned_ufc_model/config.json (deflated 45%)
  adding: content/fine_tuned_ufc_model/special_tokens_map.json (deflated 42%)
  adding: content/fine_tuned_ufc_model/model.safetensors (deflated 8%)
  adding: content/fine_tuned_ufc_model/tokenizer_config.json (deflated 75%)
  adding: content/fine_tuned_ufc_model/vocab.txt (deflated 53%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Script to Reload Model Later for Inference

from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Reload model and tokenizer
reloaded_model = DistilBertForSequenceClassification.from_pretrained("/content/fine_tuned_ufc_model")
reloaded_tokenizer = DistilBertTokenizer.from_pretrained("/content/fine_tuned_ufc_model")